# Searching the Catalog
All objects support the same search interface. `Searches` work by creating a query builder (class `Search`), which can be used in a fluent programming style to refine the search prior to execution by applying filtering, sorting, and limiting of result sets. Normally `Search` objects are created using class methods on one of the primary object types, e.g. `Product.search()`.

The searches are then executed by any of several methods: calling the `count()` method to obtain a count of matching objects, using the `Search` object in an iterating context such as a for loop or a list comprehension to yield each matching object in turn, or calling the `collect()` method which will return a list-like collection object (e.g. `ProductCollection`, `BandCollection`, or `ImageCollection`).

`Search` object methods never mutate the original object, but instead return modified copies. Thus Search objects can be reused for both further modification and repeated executions.

Let’s look at two of the most commonly searched for types of objects: products and images.

## Finding products

### Filtering, sorting, and limiting
Filtering is achieved through the use of the `Properties` class which allows you to express logical and comparison operations on attributes of an object such as a product or image. Multiple filters are combined as if by `AND`. Please see the API documentation for further details; the uses demonstrated below should be readily apparent. A general-use instance of this class can be imported from `descarteslabs.catalog.properties`.

Sorting by an attribute of an object in either ascending or descending order is supported for many of the attributes of each object type.

API documentation should be consulted to determine which properties support filtering and/or sorting. This is noted on each attribute’s specific documentation, e.g. `acquired`.

Limiting allows you to restrict search results to at most a specified number of objects.

`Product.search()` is the entry point for searching products. It returns a query builder that you can use to refine your search and can iterate over to retrieve search results.

Count all products with some data before 2023 using `filter()`:

In [ ]:
from descarteslabs.catalog import Product, properties as p

search = Product.search().filter(p.start_datetime < "2023-01-01")
search.count()

You can apply multiple filters. To restrict this search to products with data before 2023 and after 2000:

In [ ]:
search = search.filter(p.end_datetime > "2000-01-01")
search.count()

Of these, get the 3 products with the oldest data, using `sort()` and `limit()`. The search is not executed until you start retrieving results by iterating over it:

In [ ]:
oldest_search = search.sort("start_datetime").limit(3)
for result in oldest_search:
    print(result.id)

Or you can execute the search to produce a `ProductCollection` object, which works like a list with lots of additional features such as filtering, grouping, and attribute extraction:

In [ ]:
products = search.limit(5).collect()
print(products.each.id)

All attributes are documented in the [Product API reference](https://docs.descarteslabs.com/descarteslabs/catalog/docs/product.html#descarteslabs.catalog.Product), which also spells out which ones can be used to filter or sort.



### Text search
Add text search to the mix using `find_text()`. This finds all products with “landsat” in the name or description:

In [ ]:
landsat_search = search.find_text("landsat").limit(None)
for product in landsat_search:
    print(product)

### Lookup by id and object relationships
If you know a product’s id, look it up directly with `Product.get()`:

In [ ]:
landsat8_collection1 = Product.get("usgs:landsat:oli-tirs:c2:l2:v0")
landsat8_collection1

Wherever there are relationships between objects expect methods such as `Product.bands()` to find related objects. This shows the first four bands of the Landsat 8 product we looked up:

In [ ]:
for band in landsat8_collection1.bands().limit(5):
    print(band)

`Product.bands()` returns a search object that can be further refined. This shows all class bands of this Landsat 8 product, sorted by name:

In [ ]:
from descarteslabs.catalog import BandType

for band in landsat8_collection1.bands().filter(p.type == BandType.CLASS).sort("name"):
    print(band)

In a similar fashion `Product.images()` returns a search object for images belonging to the product, as detailed in the next section.

